In [2]:
from pythreejs import *
from IPython.display import display
import ipywidgets as widgets
from math import sqrt, tan
from random import random , randint , seed
phi = (1+sqrt(5))/2  # calculating phi-based-pi, piG.
piG = 4/sqrt(phi)    # piG is used instead of pi.
pRadius = 0.05 # proton radius, used by all atomic distances
caroSecs = 1 # time in secs for the carousal to make one rotation

In [3]:
view_width = 600
view_height = 400
camera = PerspectiveCamera(position=[5, 0, 0], up=[0, 0, 1], aspect=view_width/view_height)
#camera = PerspectiveCamera(position=[5, 0, 0], up=[1, 0, 0], aspect=view_width/view_height)
camera = PerspectiveCamera(position=[5, 0, 0], up=[0, 1, 0], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 5, 5])
ambient_light = AmbientLight()

In [4]:
elements = ((' 1 Hydrogen H ',1),(' 2 Helium He ',2),(' 3 Lithium Li ',3),(' 4 Beryllium Be ',4),
            (' 5 Boron B ',5),(' 6 Carbon C ',6),(' 7 Nitrogen N ',7),(' 8 Oxygen O ',8),
            (' 9 Fluorine F ',9),(' 10 Neon Ne ',10),(' 11 Sodium Na ',11),(' 12 Magnesium Mg ',12),
            (' 13 Aluminum Al ',13),(' 14 Silicon Si ',14),(' 15 Phosphorus P ',15),(' 16 Sulfur S ',16),
            (' 17 Chlorine Cl ',17),(' 18 Argon Ar ',18),(' 19 Potassium P ',19),(' 20 Calcium Ca ',20),
            (' 21 Scandium Sc ',21),(' 22 Titanium Ti ',22),(' 23 Vanadium V ',23),(' 24 Chromium Cr ',24),
            (' 25 Manganese Mn ',25),(' 26 Iron Fe ',26),(' 27 Cobalt Co ',27),(' 28 Nickel Ni ',28),
            (' 29 Copper Cu ',29),(' 30 Zinc Zn ',30),(' 31 Gallium Ga ',31),(' 32 Germanium Ge ',32),
            (' 33 Arsenic As ',33),(' 34 Selenium Se ',34),(' 35 Bromine Br ',35),(' 36 Krypton Kr ',36),
            (' 37 Rubidium Rb ',37),(' 38 Strontium Sr ',38),(' 39 Yttrium Y ',39),(' 40 Zirconium Zr ',40),
            (' 41 Niobium Nb ',41),(' 42 Molybdenum Mo ',42),(' 43 Technetium Tc ',43),(' 44 Ruthenium Ru ',44),
            (' 45 Rhodium Rh ',45),(' 46 Palladium Pd ',46),(' 47 Silver Ag ',47),(' 48 Cadmium Cd ',48),
            (' 49 Indium In ',49),(' 50 Tin Sn ',50),(' 51 Antimony Sb ',51),(' 52 Tellurium Te ',52),
            (' 53 Iodine I ',53),(' 54 Xenon Xe ',54),(' 55 Caesium Cs ',55),(' 56 Barium Ba ',56),
            (' 57 Lanthanum La ',57),(' 58 Cerium Ce ',58),(' 59 Praseodymium Pr ',59),(' 60 Neodymium Nd ',60),
            (' 61 Promethium Pm ',61),(' 62 Samarium Sm ',62),(' 63 Europium Eu ',63),(' 64 Gadolinium Gd ',64),
            (' 65 Terbium Tb ',65),(' 66 Dysprosium Dy ',66),(' 67 Holmium Ho ',67),(' 68 Erbium Er ',68),
            (' 69 Thulium Tm ',69),(' 70 Ytterbium Yb ',70),(' 71 Lutetium Lu ',71),(' 72 Hafnium Hf ',72),
            (' 73 Tantalum Ta ',73),(' 74 Tungsten W ',74),(' 75 Rhenium Re ',75),(' 76 Osmium Os ',76),
            (' 77 Iridium Ir ',77),(' 78 Platinum Pt ',78),(' 79 Gold Au ',79),(' 80 Mercury Hg ',80),
            (' 81 Thallium Tl ',81),(' 82 Lead Pb ',82),(' 83 Bismuth Bi ',83),(' 84 Polonium Po ',84),
            (' 85 Astatine At ',85),(' 86 Radon Rn ',86),(' 87 Francium Fr ',87),(' 88 Radium Ra ',88),
            (' 89 Actinium Ac ',89),(' 90 Thorium Th ',90));

In [5]:
selectAtom = widgets.Dropdown(
    options= elements,
    value=2, 
    description='Atom:',
)
atomsLabel = widgets.Checkbox(
    value=False,
    description='Atomic label',
    disabled=False,
    indent=False
)
emissionType= widgets.Dropdown(
    options=['plane', 'cones'],
    value='plane', 
    description='Emission:',
)
coloredEmissions = widgets.Checkbox(
    value=False,
    description='Color emissions',
    disabled=False,
    indent=False,
)
# Changes the distance between adjacent orthogonal protons
emRSlider = widgets.FloatSlider(
    value = 5.0,
    min = 2.0,
    max = 8.0,
    step= 0.1,
    description='EmRadius',
)
# Changes the distance between adjacent orthogonal protons
ePSlider = widgets.FloatSlider(
    value = 5.0,
    min = 3.0,
    max = 7.0,
    step= 0.1,
    description='Equ1_Pole2',
)
# Changes the distance tween top and bottom alpha parallel protons
pPSlider = widgets.FloatSlider(
    value = 4.0,
    min = 3.0,
    max = 6.0,
    step= 0.1,
    description='A width',
)
# Changes the pole to pole distance between 2 adjacent alphas
p2p3Slider = widgets.FloatSlider(
    value = 3.0,
    min = 2.0,
    max = 5.0,
    step= 0.1,
    description='A1_A2 Sep',
)
# Changes the cone emission angle
coneAngleSlider = widgets.FloatSlider(
    value = 30,  # deg  
    min = 20,   
    max = 40,  
    step= 0.1,
    description='Cone angle',
)
widgets.VBox([selectAtom,atomsLabel,emissionType,
              coloredEmissions,emRSlider,ePSlider,
              pPSlider,p2p3Slider,coneAngleSlider])

In [6]:
number = selectAtom.value
def handle_change(change):
    global number
    number = change.new
selectAtom.observe(handle_change, 'value')

emR = emRSlider.value
def handle_change(change):
    global emR
    emR = change.new
emRSlider.observe(handle_change, 'value');

ortho = ePSlider.value # proton1 equator to proton2 pole
def handle_change(change):
    global ortho
    ortho = change.new
ePSlider.observe(handle_change, 'value');

pPTP = pPSlider.value  # proton pole to proton pole 
def handle_change(change):
    global pPTP
    pPTP = change.new
pPSlider.observe(handle_change, 'value')

p2p3Dist = p2p3Slider.value # between 2 adjacent alphas
def handle_change(change):
    global p2p3Dist
    p2p3Dist = change.new
p2p3Slider.observe(handle_change, 'value')

cAngleDeg = coneAngleSlider.value # cone angle
def handle_change(change):
    global cAngleDeg
    cAngleDeg = change.new
coneAngleSlider.observe(handle_change, 'value')


In [7]:
An = elements[number-1]
atomicLabel = An[0]
print(atomicLabel) # used to verify An control changes, eliminate this line at any time.
print(emissionType.value)

 23 Vanadium V 
cones


In [8]:
atomicNumber = number  # Select the desired atom from the dropdown widget
grpAn = Group()  # The rotation widget spins grpAn, slots 1,4 and 5.
grpBn = Group()  # grpBn are the orthogonal top, bottom and hook slots.

if 1 <= number < 13:
    caroSecs = 2.5
if 14 <= number < 20:
    caroSecs = 4.0
if 21 <= number :
    caroSecs = 6.0
    
# Dimensions # pRadius is now in In[1]
nRadius = pRadius*1.125 # Neutron radius
eRadius = 2*pRadius/10 # Electron radius

# Widget Controlled variables 1. ortho, 2.pPTP, 3. p2p3Dist, 4. cAngleDeg
orthoP = ortho*pRadius
emRadius = 2*emR*pRadius # 2*orthoP - pRadius/2
coneAngle = cAngleDeg*2*piG/360
coneHeight = emRadius*tan(coneAngle)
pP1pP2 = 2*pPTP*pRadius # Widget control.
p2p3 = 2*p2p3Dist*pRadius # Widget control
n1n2 = (pP1pP2- 2*nRadius)/tan(coneAngle) # neutron-neutron orbital separation (y) inside the alpha
# e1e2 = -n1n2/10
p1p2 = pRadius*2.5  #0.75*pP1pP2 # pRadius*2.5
p1n1 = pP1pP2/2 #pP1pP2/2 #pP1pP2/2 # Alpha, up-down (z) proton-orbital neutron separation
p1n2 = pP1pP2/2 #pRadius + nRadius + 1.5*pRadius  # In line proton and neutron
p1e1 = pRadius + 0.05*pP1pP2 # Alpha, top-bottom (z) proton-electron separation
maxZNeg = 0.0 # maxZNeg and maxZPos are Used to center(z) the atom on the screen
maxZPos = 0.0

# Proton set Geometries and Materials
protonGeometry =  SphereBufferGeometry(pRadius, 24, 16)
neutronGeometry =  SphereBufferGeometry(nRadius, 24, 16)
electronGeometry =  SphereBufferGeometry(eRadius, 8, 16)
emissionGeometryP = CircleGeometry(emRadius, 24)
emissionGeometryX = ConeGeometry( emRadius, coneHeight, 16, openEnded = True )
protonMaterial = MeshStandardMaterial(color='blue')
protonMaterialB = MeshStandardMaterial(color='green')
neutronMaterial = MeshStandardMaterial(color='red')
neutronMaterialB = MeshStandardMaterial(color='green') # Color.js Find a nice color spectrum
electronMaterial = MeshStandardMaterial(color='cyan') #'orangered','salmon','green','cyan','teal','orange'
emissionmaterial7= MeshStandardMaterial(color='gray',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial1 = MeshStandardMaterial(color='magenta',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial2 = MeshStandardMaterial(color='blue',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial3 = MeshStandardMaterial(color='pink',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial4 = MeshStandardMaterial(color='salmon',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial5 = MeshStandardMaterial(color='lime',side='DoubleSide',transparent = True,opacity = 0.20)
emissionmaterial6 = MeshStandardMaterial(color='skyblue',side='DoubleSide',transparent = True,opacity = 0.20)

def buildProtonStack(a,j): #  a is the total # of protons. j indicates the calling register
    """ The odd 'a' returns just the single, first, third, or fifth proton. The even 'a'
     returns the first, second or third alpha set, each 2 protons and 2 neutrons. """
    num = 0
    if coloredEmissions.value == True:
        if a == 1 : emissionmaterial = emissionmaterial1
        if a == 2 : emissionmaterial = emissionmaterial2
        if a == 3 : emissionmaterial = emissionmaterial3
        if a == 4 : emissionmaterial = emissionmaterial4
        if a == 5 : emissionmaterial = emissionmaterial5
        if a == 6 : emissionmaterial = emissionmaterial6
    else:
        emissionmaterial = emissionmaterial7
        # Proton z positions: # The possible formula solution, rather than the list, may be in these 2 lines 
        # protonMi.position = [0,0,a*pP1pP2/2+(a-1)*p2p3/2-(p2p3/2+pP1pP2/2)*num] # multiAlpha solution - Not
        # protonMi.position = [0,0,(a//2)*pP1pP2/2+(a//2)*p2p3/2-(p2p3/2+pP1pP2/2)*num] # pSet
    if a == 1 : pPositions = [0]
    if a == 2 : pPositions = [pP1pP2/2,-pP1pP2/2]
    if a == 3 : pPositions = [pP1pP2/2+p2p3/2,-pP1pP2/2+p2p3/2,-pP1pP2/2-p2p3/2]
    if a == 4 : pPositions = [pP1pP2+p2p3/2,p2p3/2,-p2p3/2,-pP1pP2-p2p3/2]
    if a == 5 : pPositions = [pP1pP2+p2p3,p2p3,0,-pP1pP2,-pP1pP2-p2p3]
    if a == 6 : pPositions = [3*pP1pP2/2+p2p3,pP1pP2/2+p2p3,pP1pP2/2,-pP1pP2/2,-pP1pP2/2-p2p3,-3*pP1pP2/2-p2p3]
    while num < a:
        protonMi = Group()
        protonMi.add(Mesh(protonGeometry, protonMaterial))
        if emissionType.value == 'plane':
            protonMi.add(Mesh(emissionGeometry, emissionmaterial))
        if emissionType.value == 'cones':
            emission1 = Object3D()
            emission1.add(Mesh(emissionGeometry, emissionmaterial))
            emission1.position = [0,0,coneHeight/2]
            emission1.rotateX(-piG/2)
            protonMi.add(emission1)
            emission2 = Object3D()
            emission2.add(Mesh(emissionGeometry, emissionmaterial))
            emission2.position = [0,0,-coneHeight/2]
            emission2.rotateX(piG/2)
            protonMi.add(emission2)
            protonMi.add(Mesh(emissionGeometryP, emissionmaterial))
        electronMi = Object3D()
        electronMi.add(Mesh(electronGeometry, electronMaterial))
        electronMi.position = [0,-n1n2/10,-p1e1]
        protonMi.add(electronMi)
        protonMi.rotateZ(random()*2*piG)
        if num%2 != 0:
            neutronPair = Group()
            neutronMi = Object3D()
            neutronMi.add(Mesh(neutronGeometry, neutronMaterial))
            neutronMi.position = [0,-n1n2/2,-p1n1]
            neutronNi = Object3D()
            neutronNi.add(Mesh(neutronGeometry, neutronMaterial))
            neutronNi.position = [0,n1n2/2,-p1n1]
            neutronPair.add(neutronMi)
            neutronPair.add(neutronNi)
            neutronPair.rotateZ(random()*2*piG)
            protonMi.add(neutronPair)
            protonMi.rotateX(piG)
            protonMi.rotateZ(random()*2*piG)
        protonMi.position = [0,0,pPositions[num]]
        if j == 0 : proStack1.add(protonMi)
        if j == 1 : proStack2.add(protonMi)
        if j == 2 : proStack3.add(protonMi)
        if j == 3 : proStack4.add(protonMi)
        if j == 4 : proStack5.add(protonMi)
        if j == 5 : proStack6.add(protonMi)
        if j == 6 : proStack7.add(protonMi)
        if j == 7 : proStack8.add(protonMi)
        if j == 8 : proStack9.add(protonMi)
        if j == 9 : proStack10.add(protonMi)
        if j == 10 : proStack11.add(protonMi)
        if j == 11 : proStack12.add(protonMi)
        if j == 12 : proStack13.add(protonMi)
        if j == 13 : proStack14.add(protonMi)
        if j == 14 : proStack15.add(protonMi)
        if j == 15 : proStack16.add(protonMi)
        if j == 16 : proStack17.add(protonMi)
        if j == 17 : proStack18.add(protonMi)
        if j == 18 : proStack19.add(protonMi)
        num += 1
    return

# Assemble the atomic models.
for i in range( atomicNumber ):
    if emissionType.value == 'plane':
        emissionGeometry = emissionGeometryP
    if emissionType.value == 'cones':
        emissionGeometry = emissionGeometryX
    if coloredEmissions.value == True:
        emissionmaterial = emissionmaterial1
    else:
        emissionmaterial = emissionmaterial7
    neutronGi = Object3D()
    neutronGi.add(Mesh(neutronGeometry, neutronMaterial))
    neutronKi = Object3D()
    neutronKi.add(Mesh(neutronGeometry, neutronMaterial))
    neutronLi = Object3D()
    neutronLi.add(Mesh(neutronGeometry, neutronMaterial))

    if i == 0: # Slot 1
        proStack1 = Group()
        if 1 == atomicNumber: # Hydrogen. Slot 1
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack1.position = [0, 0, 0]
            proStack1.add(neutronGi)
            grpAn.add(proStack1)
            print('proStack1 1[',i+1,']', proStack1.position)
        if 2 <= atomicNumber < 6: # Helium
            buildProtonStack(2,i)
            grpAn.add(proStack1)
            print('proStack1 2[',i+1,']', proStack1.position)
        if 6 == atomicNumber :
            buildProtonStack(4,i)
            grpAn.add(proStack1)
            maxZNeg = -3*pP1pP2/2-2*orthoP
            maxZPos = 3*pP1pP2/2+2*orthoP
            print('proStack1 4[',i+1,']', proStack1.position)
        if 7 <= atomicNumber < 10 :
            buildProtonStack(6,i)
            grpAn.add(proStack1)
            maxZNeg = -3*pP1pP2/2-p2p3
            maxZPos = p2p3+3*pP1pP2/2+2*orthoP
            print('proStack1 6[',i+1,']', proStack1.position)
        if 10 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            grpAn.add(proStack1)
            print('proStack1 2[',i+1,']', proStack1.position)
        if 36 <= atomicNumber < 54 :
            buildProtonStack(4,i)
            grpAn.add(proStack1)
            print('proStack1 4[',i+1,']', proStack1.position)
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            grpAn.add(proStack1)
            print('proStack1 6[',i+1,']', proStack1.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)            
            grpAn.add(proStack1)
            print('proStack1 5[',i+1,']', proStack1.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            grpAn.add(proStack1)
            print('proStack1 6[',i+1,']', proStack1.position)

    if i == 1: # Slot 2
        proStack2 = Group()
        if 2 < atomicNumber < 6:
            buildProtonStack(1,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0, pP1pP2/2 + 2*orthoP]
            grpBn.add(proStack2)
            print('proStack2 1[',i+1,']', proStack2.position)
        if 6 == atomicNumber :
            buildProtonStack(1,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0,p2p3/2+pP1pP2+2*orthoP]
            grpBn.add(proStack2)
            print('proStack2 1[',i+1,']', proStack2.position)
        if 7 <= atomicNumber < 10:
            buildProtonStack(1,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack2)
            print('proStack2 1[',i+1,']', proStack2.position)
        if 10 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack2.rotateX(-piG/2)
            proStack2.position = [0, 0, pP1pP2/2 + 2*orthoP] 
            grpBn.add(proStack2)
            print('proStack2 2[',i+1,']', proStack2.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack2)
            print('proStack2 4[',i+1,']', proStack2.position)
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack2)
            print('proStack2 6[',i+1,']', proStack2.position)
        if 57 <= atomicNumber < 71 :
            buildProtonStack(5,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack2)
            print('proStack2 5[',i+1,']', proStack2.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack2.rotateX(piG/2)
            proStack2.position = [0,0,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack2)
            print('proStack2 6[',i+1,']', proStack2.position)

    if i == 2: # Slot 3
        proStack3 = Group()
        if 3 < atomicNumber < 5:
            buildProtonStack(1,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -pP1pP2/2 - 2*orthoP]
            grpBn.add(proStack3)
            print('proStack3 1[',i+1,']', proStack3.position)
        if 5 == atomicNumber :
            buildProtonStack(2,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -pP1pP2/2 - 2*orthoP]
            grpBn.add(proStack3)
            print('proStack3 2[',i+1,']', proStack3.position)
        if 6 == atomicNumber :
            buildProtonStack(1,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -p2p3/2-pP1pP2-2*orthoP]
            grpBn.add(proStack3)
            print('proStack3 1[',i+1,']', proStack3.position)
        if 8 == atomicNumber :
            buildProtonStack(1,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0,-3*pP1pP2/2-p2p3-2*orthoP]
            grpBn.add(proStack3)
            maxZNeg = -3*pP1pP2/2-p2p3-2*orthoP
            maxZPos = 3*pP1pP2/2+p2p3+2*orthoP
            print('proStack3 1[',i+1,']', proStack3.position)
        if 9 == atomicNumber :
            buildProtonStack(2,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0,-3*pP1pP2/2-p2p3-2*orthoP]
            grpBn.add(proStack3)
            maxZNeg = -3*pP1pP2/2-p2p3-2*orthoP
            maxZPos = 3*pP1pP2/2+p2p3+2*orthoP
            print('proStack3 2[',i+1,']', proStack3.position)
        if 10 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack3.rotateX(-piG/2)
            proStack3.position = [0, 0, -pP1pP2/2 - 2*orthoP]
            grpBn.add(proStack3)
            print('proStack3 2[',i+1,']', proStack3.position)
        if 36 <= atomicNumber < 54 :
            buildProtonStack(4,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack3)
            print('proStack3 4[',i+1,']', proStack3.position)
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack3)
            print('proStack3 6[',i+1,']', proStack3.position)
        if 57 <= atomicNumber < 71 :
            buildProtonStack(5,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack3)
            print('proStack3 5[',i+1,']', proStack3.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack3.rotateX(piG/2)
            proStack3.position = [0,0,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack3)
            print('proStack3 6[',i+1,']', proStack3.position)

    if i == 3: # Slot 4
        proStack4 = Group()
        if 10 <= atomicNumber < 14: # Neon
            buildProtonStack(2,i)
            proStack4.position = [0, 0, pP1pP2 + 4*orthoP]
            grpBn.add(proStack4)
            print('proStack4 2[',i+1,']', proStack4.position)
        if 15 == atomicNumber :
            buildProtonStack(2,i)
            proStack4.position = [0, 0, pP1pP2 + 4*orthoP]
            grpBn.add(proStack4)
            print('proStack4 2[',i+1,']', proStack4.position)
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+4*orthoP
        if 16 == atomicNumber :
            buildProtonStack(1,i)
            proStack4.position = [0, 0, pP1pP2/2 + 4*orthoP]
            grpBn.add(proStack4)
            print('proStack4 1[',i+1,']', proStack4.position)
            maxZNeg = -pP1pP2/2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 17 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack4.position = [0, 0, pP1pP2 + 4*orthoP]
            grpBn.add(proStack4)
            print('proStack4 2[',i+1,']', proStack4.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack4.position = [0,0,p1p2/2+pP1pP2+p1p2/2+pP1pP2+4*orthoP]
            grpBn.add(proStack4)
            print('proStack4 4[',i+1,']', proStack4.position)
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack4.position = [0,0,2*p2p3+3*pP1pP2+4*orthoP]
            grpBn.add(proStack4)
            print('proStack4 6[',i+1,']', proStack4.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack4.position = [0,0,2*p2p3+2*pP1pP2+4*orthoP]
            grpBn.add(proStack4)
            print('proStack4 5[',i+1,']', proStack4.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack4.position = [0,0,2*p2p3+3*pP1pP2+4*orthoP]
            grpBn.add(proStack4)
            print('proStack4 6[',i+1,']', proStack4.position)

    if i == 4: # Slot 5
        proStack5 = Group()
        if 10 <= atomicNumber < 14 :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            grpBn.add(proStack5)
            print('proStack5 2[',i+1,']', proStack5.position)
        if 15 == atomicNumber :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            grpBn.add(proStack5)
            print('proStack5 2[',i+1,']', proStack5.position)
        if 16 == atomicNumber :
            buildProtonStack(1,i)
            proStack5.position = [0, 0, -pP1pP2/2 - 4*orthoP]
            grpBn.add(proStack5)
            print('proStack5 1[',i+1,']', proStack5.position)
            maxZNeg = -pP1pP2/2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 17 <= atomicNumber < 21 :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            grpBn.add(proStack5)
            print('proStack5 2[',i+1,']', proStack5.position)
        if 21 == atomicNumber :
            buildProtonStack(2,i)
            neutronGi.position = [0, 0, -pP1pP2/2 - p1n2]
            proStack5.add(neutronGi)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            grpBn.add(proStack5)
            print('proStack5 2 w neutron[',i+1,']', proStack5.position)
        if 22 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack5.position = [0, 0, -pP1pP2 - 4*orthoP]
            grpBn.add(proStack5)
            print('proStack5 2[',i+1,']', proStack5.position)
        if 36 <= atomicNumber < 54:
            proStack5 = Group()
            buildProtonStack(4,i)
            proStack5.position = [0,0,-p1p2/2-pP1pP2-p1p2/2-pP1pP2-4*orthoP]
            grpBn.add(proStack5)
            print('proStack5 4[',i+1,']', proStack5.position)
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack5.position = [0,0,-2*p2p3-3*pP1pP2-4*orthoP]
            grpBn.add(proStack5)
            print('proStack5 6[',i+1,']', proStack5.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack5.position = [0,0,-2*p2p3-2*pP1pP2-4*orthoP]
            grpBn.add(proStack5)
            print('proStack5 5[',i+1,']', proStack5.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack5.position = [0,0,-2*p2p3-3*pP1pP2-4*orthoP]
            grpBn.add(proStack5)
            print('proStack5 6[',i+1,']', proStack5.position)

    if i == 5: # Slot 6
        proStack6 = Group()
        if 11 <= atomicNumber  < 14 :
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0,0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 1[',i+1,']', proStack6.position)
        if 17 <= atomicNumber < 18 :
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 2[',i+1,']', proStack6.position)
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 19 <= atomicNumber < 22 :
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 1[',i+1,']', proStack6.position)
            maxZNeg = -3*pP1pP2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 23 <= atomicNumber < 26 :
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 1[',i+1,']', proStack6.position)
            maxZNeg = -3*pP1pP2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 26 <= atomicNumber < 29:
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 2[',i+1,']', proStack6.position)
        if 29 <= atomicNumber < 31:
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 2[',i+1,']', proStack6.position)
            maxZNeg = -3*pP1pP2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 31 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 2[',i+1,']', proStack6.position)
        if 37 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0,0,3*p1p2/2+6*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 1[',i+1,']', proStack6.position)
            maxZNeg = -3*p1p2/2-5*pP1pP2/2-4*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 48 <= atomicNumber < 50:
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*p1p2/2+6*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 2[',i+1,']', proStack6.position)
        if 50 == atomicNumber:
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0,0,3*p1p2/2+6*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 1[',i+1,']', proStack6.position)
        if 51 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0, 3*p1p2/2+6*pP1pP2/2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 2[',i+1,']', proStack6.position)
        if 55 <= atomicNumber < 57 :
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 1[',i+1,']', proStack6.position)
            maxZNeg = -3*p1p2/2-5*pP1pP2/2-4*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 57 <= atomicNumber < 62:
            buildProtonStack(1,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+3*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 1[',i+1,']', proStack6.position)
        if 62 <= atomicNumber < 66 :
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+3*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 2[',i+1,']', proStack6.position)
        if 66 <= atomicNumber < 71:
            buildProtonStack(3,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+3*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 3[',i+1,']', proStack6.position)
        if 71 == atomicNumber :
            buildProtonStack(2,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 2[',i+1,']', proStack6.position)
        if 72 <= atomicNumber < 78 :
            buildProtonStack(3,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 3[',i+1,']', proStack6.position)
        if 78 == atomicNumber :
            buildProtonStack(4,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 4[',i+1,']', proStack6.position)
        if 79 == atomicNumber :
            buildProtonStack(3,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 3[',i+1,']', proStack6.position)
        if 80 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 4[',i+1,']', proStack6.position)
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack6.rotateX(-piG/2)
            proStack6.position = [0, 0,3*p2p3+4.5*pP1pP2+6*orthoP]
            grpBn.add(proStack6)
            print('proStack6 5[',i+1,']', proStack6.position)

    if i == 6: # Slot 7
        proStack7 = Group()
        if 12 == atomicNumber :
            buildProtonStack(1,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0,0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 1[',i+1,']', proStack7.position)
        if 15 == atomicNumber:
            buildProtonStack(1,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 1[',i+1,']', proStack7.position)
        if 17 == atomicNumber:
            buildProtonStack(1,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 1[',i+1,']', proStack7.position)
        if 20 == atomicNumber:
            buildProtonStack(1,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
            print('proStack7 1[',i+1,']', proStack7.position)
        if 24 <= atomicNumber < 26:
            buildProtonStack(1,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
            print('proStack7 1[',i+1,']', proStack7.position)
        if 26 <= atomicNumber < 30 :
            buildProtonStack(2,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 2[',i+1,']', proStack7.position)
        if 30 == atomicNumber :
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            maxZNeg = -3*pP1pP2/2-6*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
            print('proStack7 1[',i+1,']', proStack7.position)
        if 31 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0, 0, -3*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 2[',i+1,']', proStack7.position)
        if 38 == atomicNumber:
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
            print('proStack7 1[',i+1,']', proStack7.position)
        if 40 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
            print('proStack7 1[',i+1,']', proStack7.position)
        if 47 <= atomicNumber < 50:
            buildProtonStack(2,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 2[',i+1,']', proStack7.position)
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 50 == atomicNumber:
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 1[',i+1,']', proStack7.position)
        if 51 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0,0,-3*p1p2/2-6*pP1pP2/2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 2[',i+1,']', proStack7.position)
        if 56 == atomicNumber < 57:
            buildProtonStack(1,i)
            proStack7.rotateX(piG/2)
            proStack7.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 1[',i+1,']', proStack7.position)
            maxZNeg = -3*p1p2/2-6*pP1pP2/2-6*orthoP # needed
            maxZPos = 3*p1p2/2+6*pP1pP2/2+6*orthoP
        if 57 <= atomicNumber < 63:
            buildProtonStack(2,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 2[',i+1,']', proStack7.position)
        if 63 <= atomicNumber < 66 :
            buildProtonStack(3,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 3[',i+1,']', proStack7.position)
        if 66 == atomicNumber :
            buildProtonStack(4,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 4[',i+1,']', proStack7.position)
        if 67 == atomicNumber :
            buildProtonStack(3,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 3[',i+1,']', proStack7.position)
        if 68 <= atomicNumber < 71 :
            buildProtonStack(4,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-3*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 4[',i+1,']', proStack7.position)
        if 71 <= atomicNumber < 78 :
            buildProtonStack(3,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 3[',i+1,']', proStack7.position)
        if 78 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 4[',i+1,']', proStack7.position)
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack7.rotateX(-piG/2)
            proStack7.position = [0, 0,-3*p2p3-4.5*pP1pP2-6*orthoP]
            grpBn.add(proStack7)
            print('proStack7 5[',i+1,']', proStack7.position)

    if i == 7: # Slot 8
        proStack8 = Group()
        if 13 == atomicNumber :
            buildProtonStack(1,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0, 2*orthoP, 0]
            grpAn.add(proStack8)
            print('proStack8 1[',i+1,']', proStack8.position)
        if 14 == atomicNumber :
            buildProtonStack(2,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,pP1pP2/2+2*orthoP, 0]
            grpAn.add(proStack8)
            print('proStack8 2[',i+1,']', proStack8.position)
        if 15 == atomicNumber :
            buildProtonStack(1,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0, 2*orthoP, 0]
            grpAn.add(proStack8)
            print('proStack8 1[',i+1,']', proStack8.position)
        if 16 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,pP1pP2/2+2*orthoP, 0]
            grpAn.add(proStack8)
            print('proStack8 2[',i+1,']', proStack8.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,p1p2/2+pP1pP2+2*orthoP,0]
            grpAn.add(proStack8)
            print('proStack8 4[',i+1,']', proStack8.position)
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,p2p3+3*pP1pP2/2+2*orthoP, 0]
            grpAn.add(proStack8)
            print('proStack8 6[',i+1,']', proStack8.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,p2p3+pP1pP2+2*orthoP,0]
            grpAn.add(proStack8)
            print('proStack8 5[',i+1,']', proStack8.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack8.rotateX(piG/2)
            proStack8.position = [0,p2p3+3*pP1pP2/2+2*orthoP, 0]
            grpAn.add(proStack8)
            print('proStack8 6[',i+1,']', proStack8.position)

    if i == 8: # Slot 9
        proStack9 = Group()
        if 13 == atomicNumber :
            buildProtonStack(1,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0, -2*orthoP, 0]
            grpAn.add(proStack9)
            print('proStack9 1[',i+1,']', proStack9.position)
            maxZNeg = -pP1pP2/2-4*orthoP
            maxZPos = 3*pP1pP2/2+6*orthoP
        if 14 == atomicNumber :
            buildProtonStack(2,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-pP1pP2/2-2*orthoP, 0]
            grpAn.add(proStack9)
            print('proStack9 2[',i+1,']', proStack9.position)
        if 15 == atomicNumber :
            buildProtonStack(1,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0, -2*orthoP, 0]
            grpAn.add(proStack9)
            print('proStack9 1[',i+1,']', proStack9.position)
        if 16 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-pP1pP2/2-2*orthoP, 0]
            grpAn.add(proStack9)
            print('proStack9 2[',i+1,']', proStack9.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-2*orthoP-p1p2/2-pP1pP2,0]
            grpAn.add(proStack9)
            print('proStack9 4[',i+1,']', proStack9.position)
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-p2p3-3*pP1pP2/2-2*orthoP, 0]
            grpAn.add(proStack9)
            print('proStack9 6[',i+1,']', proStack9.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-p2p3-pP1pP2-2*orthoP,0]
            grpAn.add(proStack9)
            print('proStack9 5[',i+1,']', proStack9.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack9.rotateX(piG/2)
            proStack9.position = [0,-p2p3-3*pP1pP2/2-2*orthoP, 0]
            grpAn.add(proStack9)
            print('proStack9 6[',i+1,']', proStack9.position)

    if i == 9: # Slot 10
        proStack10 = Group()
        if 14 == atomicNumber:
            buildProtonStack(2,i)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [pP1pP2/2+2*orthoP, 0, 0]
            grpAn.add(proStack10)
            print('proStack10 2[',i+1,']', proStack10.position)
        if 15 == atomicNumber :
            buildProtonStack(2,i)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [2*orthoP, 0, 0]
            grpAn.add(proStack10)
            print('proStack10 2[',i+1,']', proStack10.position)
        if 16 == atomicNumber:
            buildProtonStack(2,i)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [pP1pP2/2+2*orthoP, 0, 0]
            grpAn.add(proStack10)
            print('proStack10 2[',i+1,']', proStack10.position)
        if 18 <= atomicNumber < 21 :
            buildProtonStack(2,i)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [pP1pP2/2+2*orthoP, 0, 0]
            grpAn.add(proStack10)
            print('proStack10 2[',i+1,']', proStack10.position)
        if 21 == atomicNumber :
            buildProtonStack(2,i)
            neutronGi.position = [0, 0, pP1pP2/2 + p1n2]
            proStack10.add(neutronGi)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [pP1pP2/2+2*orthoP, 0, 0]
            grpAn.add(proStack10)
            print('proStack10 2 w neutron[',i+1,']', proStack10.position)
        if 22 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack10.rotateX(-piG/2)
            proStack10.rotateY(piG/2)
            proStack10.position = [pP1pP2/2+2*orthoP, 0, 0]
            grpAn.add(proStack10)
            print('proStack10 2[',i+1,']', proStack10.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack10.rotateY(piG/2)
            proStack10.position = [2*orthoP+p1p2/2+pP1pP2,0,0]
            grpAn.add(proStack10)
            print('proStack10 4[',i+1,']', proStack10.position)
        if 54 <= atomicNumber < 57 :
            buildProtonStack(6,i)
            proStack10.rotateY(piG/2)
            proStack10.position = [p2p3+3*pP1pP2/2+2*orthoP,0,0]
            grpAn.add(proStack10)
            print('proStack10 6[',i+1,']', proStack10.position)
        if 57 <= atomicNumber < 71:
            buildProtonStack(5,i)
            proStack10.rotateY(piG/2)
            proStack10.position = [p2p3+pP1pP2+2*orthoP,0,0]
            grpAn.add(proStack10)
            print('proStack10 5[',i+1,']', proStack10.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack10.rotateY(piG/2)
            proStack10.position = [p2p3+3*pP1pP2/2+2*orthoP,0,0]
            grpAn.add(proStack10)
            print('proStack10 6[',i+1,']', proStack10.position)

    if i == 10: # Slot 11
        proStack11 = Group()
        if 14 == atomicNumber: # Silicon, the carousal level
            buildProtonStack(2,i)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-pP1pP2/2-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            print('proStack11 2[',i+1,']', proStack11.position)
        if 15 == atomicNumber:
            buildProtonStack(2,i)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            print('proStack11 2[',i+1,']', proStack11.position)
        if 16 == atomicNumber:
            buildProtonStack(2,i)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-pP1pP2/2-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            print('proStack11 2[',i+1,']', proStack11.position)
        if 18 <= atomicNumber < 21 :
            buildProtonStack(2,i)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-pP1pP2/2-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            print('proStack11 2[',i+1,']', proStack11.position)
        if 21 == atomicNumber :
            buildProtonStack(2,i)
            neutronGi.position = [0, 0, -pP1pP2/2 - p1n2]
            proStack11.add(neutronGi)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-pP1pP2/2-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            print('proStack11 2 w neutron[',i+1,']', proStack11.position)
        if 22 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack11.rotateX(-piG/2)
            proStack11.rotateY(piG/2)
            proStack11.position = [-pP1pP2/2-2*orthoP, 0, 0]
            grpAn.add(proStack11)
            print('proStack11 2[',i+1,']', proStack11.position)
        if 36 <= atomicNumber < 54:
            buildProtonStack(4,i)
            proStack11.rotateY(piG/2)
            proStack11.position = [-2*orthoP-p1p2/2-pP1pP2,0,0]
            grpAn.add(proStack11)
            print('proStack11 4[',i+1,']', proStack11.position)
        if 54 <= atomicNumber < 57:
            buildProtonStack(6,i)
            proStack11.rotateY(piG/2)
            proStack11.position = [-p2p3-3*pP1pP2/2-2*orthoP,0,0]
            grpAn.add(proStack11)
            print('proStack11 6[',i+1,']', proStack11.position)
        if 57 <= atomicNumber < 71 :
            buildProtonStack(5,i)
            proStack11.rotateY(piG/2)
            proStack11.position = [-p2p3-pP1pP2-2*orthoP,0,0]
            grpAn.add(proStack11)
            print('proStack11 5[',i+1,']', proStack11.position)
        if 71 <= atomicNumber :
            buildProtonStack(6,i)
            proStack11.rotateY(piG/2)
            proStack11.position = [-p2p3-3*pP1pP2/2-2*orthoP,0,0]
            grpAn.add(proStack11)
            print('proStack11 6[',i+1,']', proStack11.position)

    if i == 11: # Slot 12
        proStack12 = Group()
        if 21 <= atomicNumber < 27:
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack12.add(neutronGi)
            proStack12.rotateZ(piG/2)
            proStack12.position = [0, pP1pP2 + 4*orthoP, 0]
            grpAn.add(proStack12)
            print('proStack12 w neutron 1[',i+1,']', proStack12.position)
        if 27 == atomicNumber:
            neutronGi.position = [0,0,0]
            proStack12.add(neutronGi)
            proStack12.position = [0,p1n2+pP1pP2+2*orthoP,0]
            grpAn.add( proStack12 )
            print('proStack12 neutron[',i+1,']', proStack12.position)
        if 28 == atomicNumber :
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack12.add(neutronGi)
            proStack12.rotateZ(piG/2)
            proStack12.position = [0, pP1pP2+4*orthoP,0]
            grpAn.add(proStack12)
            print('proStack12 w neutron 1[',i+1,']', proStack12.position)
        if 29 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack12.rotateZ(piG/2)
            proStack12.position = [0,pP1pP2+4*orthoP,0]
            grpAn.add(proStack12)
            print('proStack12 2[',i+1,']', proStack12.position)
        if 42 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack12.position = [0,4*pP1pP2/2+p1p2+4*orthoP,0]
            grpAn.add(proStack12)
            print('proStack12 1[',i+1,']', proStack12.position)
        if 47 <= atomicNumber < 54 :
            buildProtonStack(2,i)
            proStack12.position = [0,4*pP1pP2/2+p1p2+4*orthoP,0]
            grpAn.add(proStack12)
            print('proStack12 2[',i+1,']', proStack12.position)
        if 57 <= atomicNumber < 61:
            buildProtonStack(1,i)
            proStack12.position = [0,2*p2p3+2*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            print('proStack12 1[',i+1,']', proStack12.position)
        if 61 <= atomicNumber < 67:
            buildProtonStack(2,i)
            proStack12.position = [0, 2*p2p3+2*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            print('proStack12 2[',i+1,']', proStack12.position)
        if 67 == atomicNumber :
            buildProtonStack(3,i)
            proStack12.position = [0, 2*p2p3+2*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            print('proStack12 3[',i+1,']', proStack12.position)
        if 68 <= atomicNumber < 71 :
            buildProtonStack(4,i)
            proStack12.position = [0, 2*p2p3+2*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            print('proStack12 4[',i+1,']', proStack12.position)
        if 71 == atomicNumber :
            buildProtonStack(2,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            print('proStack12 2[',i+1,']', proStack12.position)
        if 72 <= atomicNumber < 79:
            buildProtonStack(3,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            print('proStack12 3[',i+1,']', proStack12.position)
        if 79 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            print('proStack12 4[',i+1,']', proStack12.position)
        if 82 <= atomicNumber < 86:
            buildProtonStack(3,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            print('proStack12 3[',i+1,']', proStack12.position)
        if 86 <= atomicNumber :
            buildProtonStack(5,i)
            proStack12.position = [0, 2*p2p3+3*pP1pP2+4*orthoP, 0]
            grpAn.add(proStack12)
            print('proStack12 5[',i+1,']', proStack12.position)

    if i == 12: # Slot 13
        proStack13 = Group()
        if 21 <= atomicNumber < 29:
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack13.add(neutronGi)
            proStack13.rotateZ(piG/2)
            proStack13.position = [0, -pP1pP2 - 4*orthoP, 0]
            grpAn.add(proStack13)
            print('proStack13 w neutron 1[',i+1,']', proStack13.position)
        if 29 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack13.rotateZ(piG/2)
            proStack13.position = [0,-pP1pP2-4*orthoP,0]
            grpAn.add(proStack13)
            print('proStack13 2[',i+1,']', proStack13.position)
        if 41 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack13.rotateZ(piG/2)
            proStack13.position = [0,-4*pP1pP2/2-p1p2-4*orthoP,0]
            grpAn.add(proStack13)
            print('proStack13 1[',i+1,']', proStack13.position)
        if 47 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack13.position = [0,-4*pP1pP2/2-p1p2-4*orthoP,0]
            grpAn.add(proStack13)
            print('proStack13 2[',i+1,']', proStack13.position)
        if 57 <= atomicNumber < 61:
            buildProtonStack(1,i)
            proStack13.rotateZ(piG/2)
            proStack13.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            print('proStack13 1[',i+1,']', proStack13.position)
        if 61 <= atomicNumber < 67:
            buildProtonStack(2,i)
            proStack13.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            print('proStack13 2[',i+1,']', proStack13.position)
        if 67 <= atomicNumber < 68:
            buildProtonStack(3,i)
            proStack13.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            print('proStack13 3[',i+1,']', proStack13.position)
        if 68 <= atomicNumber < 71 :
            buildProtonStack(4,i)
            proStack13.position = [0, -2*p2p3-2*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            print('proStack13 4[',i+1,']', proStack13.position)
        if 71 <= atomicNumber < 79:
            buildProtonStack(3,i)
            proStack13.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            print('proStack13 3[',i+1,']', proStack13.position)
        if 79 <= atomicNumber < 82 :
            buildProtonStack(4,i)
            proStack13.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            print('proStack13 4[',i+1,']', proStack13.position)
        if 82 <= atomicNumber < 86:
            buildProtonStack(3,i)
            proStack13.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            print('proStack13 3[',i+1,']', proStack13.position)
        if 86 <= atomicNumber :
            buildProtonStack(5,i)
            proStack13.position = [0, -2*p2p3-3*pP1pP2-4*orthoP, 0]
            grpAn.add(proStack13)
            print('proStack13 5[',i+1,']', proStack13.position)

    if i == 13: # Slot 14
        proStack14 = Group()
        if 22 <= atomicNumber < 27:
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack14.add(neutronGi)
            proStack14.rotateZ(-piG/2)
            proStack14.position = [4*orthoP+pP1pP2,0,0]
            grpAn.add(proStack14)
            print('proStack14 1 w neutron[',i+1,']', proStack14.position)
        if 27 <= atomicNumber < 36:
            buildProtonStack(2,i)
            proStack14.rotateZ(-piG/2)
            proStack14.position = [4*orthoP+pP1pP2,0,0]
            grpAn.add(proStack14)
            print('proStack14 2[',i+1,']', proStack14.position)
        if 39 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack14.position = [4*pP1pP2/2+p1p2+4*orthoP,0,0]
            grpAn.add(proStack14)
            print('proStack14 1[',i+1,']', proStack14.position)
        if 48 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack14.position = [4*pP1pP2/2+p1p2+4*orthoP,0,0]
            grpAn.add(proStack14)
            print('proStack14 2[',i+1,']', proStack14.position)
        if 57 <= atomicNumber < 63:
            buildProtonStack(2,i)
            proStack14.position = [2*p2p3+2*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            print('proStack14 2[',i+1,']', proStack14.position)
        if 63 <= atomicNumber < 71:
            buildProtonStack(3,i)
            proStack14.position = [2*p2p3+2*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            print('proStack14 3[',i+1,']', proStack14.position)
        if 71 <= atomicNumber < 74:
            buildProtonStack(2,i)
            proStack14.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            print('proStack14 2[',i+1,']', proStack14.position)
        if 74 <= atomicNumber < 80:
            buildProtonStack(3,i)
            proStack14.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            print('proStack14 3[',i+1,']', proStack14.position)
        if 80 <= atomicNumber < 82:
            buildProtonStack(4,i)
            proStack14.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            print('proStack14 4[',i+1,']', proStack14.position)
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack14.position = [2*p2p3+3*pP1pP2+4*orthoP,0,0]
            grpAn.add(proStack14)
            print('proStack14 5[',i+1,']', proStack14.position)

    if i == 14: # Slot 15
        proStack15 = Group()
        if 22 <= atomicNumber < 27 :
            buildProtonStack(1,i)
            neutronGi.position = [0,0,-p1n2]
            proStack15.add(neutronGi)
            proStack15.rotateZ(-piG/2)
            proStack15.position = [-4*orthoP-pP1pP2,0,0]
            grpAn.add(proStack15)
            print('proStack15 1 w neutron[',i+1,']', proStack15.position)
        if 27 <= atomicNumber < 36 :
            buildProtonStack(2,i)
            proStack15.rotateZ(-piG/2)
            proStack15.position = [-4*orthoP-pP1pP2,0,0]
            grpAn.add(proStack15)
            print('proStack15 2[',i+1,']', proStack15.position)
        if 39 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack15.position = [-4*pP1pP2/2-p1p2-4*orthoP,0,0]
            grpAn.add(proStack15)
            print('proStack15 1[',i+1,']', proStack15.position)
        if 48 <= atomicNumber < 54:
            buildProtonStack(2,i)
            proStack15.position = [-4*pP1pP2/2-p1p2-4*orthoP,0,0]
            grpAn.add(proStack15)
            print('proStack15 2[',i+1,']', proStack15.position)
        if 57 <= atomicNumber < 63 :
            buildProtonStack(2,i)
            proStack15.position = [-2*p2p3-2*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            print('proStack15 2[',i+1,']', proStack15.position)
        if 63 <= atomicNumber < 71 :
            buildProtonStack(3,i)
            proStack15.position = [-2*p2p3-2*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            print('proStack15 3[',i+1,']', proStack15.position)
        if 71 == atomicNumber :
            buildProtonStack(3,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP, 0, 0]
            grpAn.add(proStack15)
            print('proStack15 3[',i+1,']', proStack15.position)
        if 72 <= atomicNumber < 74 :
            buildProtonStack(2,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            print('proStack15 2[',i+1,']', proStack15.position)
        if 74 == atomicNumber :
            buildProtonStack(3,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            print('proStack15 3[',i+1,']', proStack15.position)
        if 75 == atomicNumber :
            buildProtonStack(2,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            print('proStack15 2[',i+1,']', proStack15.position)
        if 76 <= atomicNumber < 80 :
            buildProtonStack(3,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            print('proStack15 3[',i+1,']', proStack15.position)
        if 80 <= atomicNumber < 82:
            buildProtonStack(4,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            print('proStack15 4[',i+1,']', proStack15.position)
        if 82 <= atomicNumber :
            buildProtonStack(5,i)
            proStack15.position = [-2*p2p3-3*pP1pP2-4*orthoP,0,0]
            grpAn.add(proStack15)
            print('proStack15 5[',i+1,']', proStack15.position)

    if i == 15: # Slot 16
        proStack16 = Group()
        if 27 <= atomicNumber < 30:
            proStack16.add(neutronGi)
            proStack16.position = [0,pP1pP2/2+p1n2,pP1pP2/2 + 2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook neutron[',i+1,']', proStack16.position)
        if 33 <= atomicNumber < 36:
            buildProtonStack(1,i)
            proStack16.position = [0, pP1pP2/2+2*orthoP, pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 46 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack16.position = [0,p1p2/2+pP1pP2+2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 49 == atomicNumber:
            buildProtonStack(1,i)
            proStack16.position = [0,p1p2/2+pP1pP2+2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 53 == atomicNumber:
            buildProtonStack(2,i)
            proStack16.position = [0,p1p2/2+pP1pP2+2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 60 <= atomicNumber < 62:
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+pP1pP2+2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 64 <= atomicNumber < 66:
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+pP1pP2+2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 70 == atomicNumber :
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+pP1pP2+2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 72 <= atomicNumber < 74:
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 75 == atomicNumber :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 77 <= atomicNumber < 80:
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 81 == atomicNumber :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 82 == atomicNumber :
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 83 <= atomicNumber < 86 :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 2[',i+1,']', proStack16.position)
        if 87 <= atomicNumber < 89 :
            buildProtonStack(1,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 1[',i+1,']', proStack16.position)
        if 89 <= atomicNumber :
            buildProtonStack(2,i)
            proStack16.position = [0,p2p3+3*pP1pP2/2+2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack16)
            print('proStack16 hook 2[',i+1,']', proStack16.position)

    if i == 16: # Slot 17
        proStack17 = Group()
        if 26 <= atomicNumber < 30:
            proStack17.add(neutronGi)
            proStack17.position = [0,-pP1pP2/2-p1n2,-pP1pP2/2 - 2*orthoP]
            grpBn.add( proStack17 )
            print('proStack17 hook neutron[',i+1,']', proStack17.position)
        if 34 <= atomicNumber < 36:
            buildProtonStack(1,i)
            proStack17.position = [0,-pP1pP2/2- 2*orthoP,-pP1pP2/2 - 2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 45 <= atomicNumber < 48:
            buildProtonStack(1,i)
            proStack17.position = [0,-p1p2/2-pP1pP2-2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 51 == atomicNumber:
            buildProtonStack(1,i)
            proStack17.position = [0,-p1p2/2-pP1pP2-2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 53 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p1p2/2-pP1pP2-2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 59 <= atomicNumber < 63 :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 65 <= atomicNumber < 66 :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 68 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 69 == atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP] 
            grpBn.add(proStack17)
            print('proStack17 hook 2[',i+1,']', proStack17.position) 
        if 70 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-pP1pP2-2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 72 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 73 == atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 2[',i+1,']', proStack17.position)
        if 74 == atomicNumber :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 75 == atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 2[',i+1,']', proStack17.position)
        if 77 <= atomicNumber < 80 :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 2[',i+1,']', proStack17.position)
        if 81 <= atomicNumber < 84:
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 84 <= atomicNumber < 86 :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 2[',i+1,']', proStack17.position)
        if 87 <= atomicNumber < 90 :
            buildProtonStack(1,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 1[',i+1,']', proStack17.position)
        if 90 <= atomicNumber :
            buildProtonStack(2,i)
            proStack17.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack17)
            print('proStack17 hook 2[',i+1,']', proStack17.position)

    if i == 17: # Slot 18
        proStack18 = Group()
        if 25 == atomicNumber:
            buildProtonStack(1,i)
            proStack18.position = [0,-pP1pP2/2-2*orthoP, pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 26 <= atomicNumber < 29:
            proStack18.add(neutronGi)
            proStack18.position = [0,-pP1pP2/2-p1n2,pP1pP2/2 + 2*orthoP]
            grpBn.add( proStack18 )
            print('proStack18 hook neutron[',i+1,']', proStack18.position)
        if 29 <= atomicNumber < 30:
            neutronGi.position = [0,-n1n2/2,0]
            proStack18.add(neutronGi)
            neutronKi.position = [0,n1n2/2,0]
            proStack18.add(neutronKi)
            proStack18.rotateX(piG/2)
            proStack18.position = [0,-pP1pP2/2-p1n2,+pP1pP2/2+2*orthoP]
            grpBn.add( proStack18 )
            print('proStack18 w 2 hook neutrons[',i+1,']', proStack18.position)
        if 30 <= atomicNumber < 36:
            buildProtonStack(1,i)
            proStack18.position = [0,-pP1pP2/2-2*orthoP,pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 44 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 50 == atomicNumber:
            buildProtonStack(2,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 2[',i+1,']', proStack18.position)
        if 51 == atomicNumber:
            buildProtonStack(1,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 52 == atomicNumber :
            buildProtonStack(2,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 2[',i+1,']', proStack18.position)
        if 53 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p1p2/2-pP1pP2-2*orthoP,p1p2/2+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 57 <= atomicNumber < 68 :
            buildProtonStack(2,i)
            proStack18.position = [0,-p2p3-pP1pP2-2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 2[',i+1,']', proStack18.position)
        if 70 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-pP1pP2-2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 71 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 76 == atomicNumber:
            buildProtonStack(2,i)
            proStack18.position = [0,-p2p3-pP1pP2-2*orthoP,p2p3+pP1pP2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 2[',i+1,']', proStack18.position)
        if 77 == atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 80 <= atomicNumber < 81 :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 1[',i+1,']', proStack18.position)
        if 85 <= atomicNumber :
            buildProtonStack(1,i)
            proStack18.position = [0,-p2p3-3*pP1pP2/2-2*orthoP,p2p3+3*pP1pP2/2+2*orthoP]
            grpBn.add(proStack18)
            print('proStack18 hook 1[',i+1,']', proStack18.position)

    if i == 18: # Slot 1
        proStack19 = Group()
        if 26 <= atomicNumber < 29:
            proStack19.add(neutronGi)
            proStack19.position = [0,pP1pP2/2+p1n2,-pP1pP2/2 - 2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook neutron[',i+1,']', proStack19.position)
        if 29 <= atomicNumber < 30:
            neutronGi.position = [0,-n1n2/2,0]
            proStack19.add(neutronGi)
            neutronKi.position = [0,n1n2/2,0]
            proStack19.add(neutronKi)
            proStack19.rotateX(piG/2)
            proStack19.position = [0,pP1pP2/2+p1n2,-pP1pP2/2-2*orthoP]
            grpBn.add( proStack19 )
            print('proStack19 w 2 hook neutrons[',i+1,']', proStack19.position)
        if 30 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,pP1pP2/2+2*orthoP,-pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 32 <= atomicNumber < 35 : 
            buildProtonStack(1,i)
            proStack19.position = [0,pP1pP2/2+2*orthoP,-pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 35 == atomicNumber :
            buildProtonStack(2,i)
            proStack19.position = [0,pP1pP2/2+2*orthoP,-pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 43 <= atomicNumber < 47:
            buildProtonStack(1,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 50 == atomicNumber:
            buildProtonStack(2,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 51 == atomicNumber:
            buildProtonStack(1,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 52 == atomicNumber:
            buildProtonStack(2,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 53 == atomicNumber:
            buildProtonStack(1,i)
            proStack19.position = [0,p1p2/2+pP1pP2+2*orthoP,-p1p2/2-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 57 == atomicNumber < 58:
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 58 <= atomicNumber < 63:
            buildProtonStack(2,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 63 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 66 <= atomicNumber < 68 :
            buildProtonStack(2,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 68 <= atomicNumber < 71:
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+pP1pP2+2*orthoP,-p2p3-pP1pP2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 71 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 74 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 76 == atomicNumber :
            buildProtonStack(2,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 2[',i+1,']', proStack19.position)
        if 80 <= atomicNumber < 81:
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 86 == atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)
        if 88 <= atomicNumber :
            buildProtonStack(1,i)
            proStack19.position = [0,p2p3+3*pP1pP2/2+2*orthoP,-p2p3-3*pP1pP2/2-2*orthoP]
            grpBn.add(proStack19)
            print('proStack19 hook 1[',i+1,']', proStack19.position)

grpAn.position = [0,0,(-maxZNeg + maxZPos)/2 - maxZPos]
grpBn.position = [0,0,(-maxZNeg + maxZPos)/2 - maxZPos]

print(emissionType.value)
print(number)

proStack1 2[ 1 ] (0.0, 0.0, 0.0)
proStack2 2[ 2 ] (0.0, 0.0, 0.7)
proStack3 2[ 3 ] (0.0, 0.0, -0.7)
proStack4 2[ 4 ] (0.0, 0.0, 1.4)
proStack5 2[ 5 ] (0.0, 0.0, -1.4)
proStack6 1[ 6 ] (0.0, 0.0, 2.1)
proStack8 2[ 8 ] (0.0, 0.7, 0.0)
proStack9 2[ 9 ] (0.0, -0.7, 0.0)
proStack10 2[ 10 ] (0.7, 0.0, 0.0)
proStack11 2[ 11 ] (-0.7, 0.0, 0.0)
proStack12 w neutron 1[ 12 ] (0.0, 1.4, 0.0)
proStack13 w neutron 1[ 13 ] (0.0, -1.4, 0.0)
proStack14 1 w neutron[ 14 ] (1.4, 0.0, 0.0)
proStack15 1 w neutron[ 15 ] (-1.4, 0.0, 0.0)
cones
23


In [9]:
if atomsLabel.value == True:
    scene = Scene(children=[grpAn, grpBn, camera, key_light, ambient_light,
        make_text(atomicLabel,
                  height=0.25,
                  position = [-1,1,number*0.025 + 1.5])]) # move the label +z as the atom grows
else:
    scene = Scene(children=[grpAn, grpBn, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera, screenSpacePanning=True)

In [10]:
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
scene = Scene()  # initialising the scene

In [11]:
spinC_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, caroSecs], values=[0, 2*piG])
spinC_clip = AnimationClip(tracks=[spinC_track])
spinC_action = AnimationAction(AnimationMixer(grpAn), spinC_clip, grpAn)
spinC_action

#print('caroSecs = ',caroSecs)

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0., 6.], dtyp…

In [12]:
renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(5.0, 0.0, 0.0), projectionMatrix=(1.4296712803397058, …

In [12]:
spinB2_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB2_clip = AnimationClip(tracks=[spinB2_track])
spinB2_action = AnimationAction(AnimationMixer(proStack2), spinB2_clip, proStack2)
spinB2_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [13]:
spinB3_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB3_clip = AnimationClip(tracks=[spinB3_track])
spinB3_action = AnimationAction(AnimationMixer(proStack3), spinB3_clip, proStack3)
spinB3_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [14]:
spinB4_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB4_clip = AnimationClip(tracks=[spinB4_track])
spinB4_action = AnimationAction(AnimationMixer(proStack4), spinB4_clip, proStack4)
spinB4_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [15]:
spinB5_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB5_clip = AnimationClip(tracks=[spinB5_track])
spinB5_action = AnimationAction(AnimationMixer(proStack5), spinB5_clip, proStack5)
spinB5_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [16]:
spinB6_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB6_clip = AnimationClip(tracks=[spinB6_track])
spinB6_action = AnimationAction(AnimationMixer(proStack6), spinB6_clip, proStack6)
spinB6_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [17]:
spinB7_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB7_clip = AnimationClip(tracks=[spinB7_track])
spinB7_action = AnimationAction(AnimationMixer(proStack7), spinB7_clip, proStack7)
spinB7_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [18]:
spinB8_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB8_clip = AnimationClip(tracks=[spinB8_track])
spinB8_action = AnimationAction(AnimationMixer(proStack8), spinB8_clip, proStack8)
spinB8_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [19]:
spinB9_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB9_clip = AnimationClip(tracks=[spinB9_track])
spinB9_action = AnimationAction(AnimationMixer(proStack9), spinB9_clip, proStack9)
spinB9_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [20]:
spinB10_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB10_clip = AnimationClip(tracks=[spinB10_track])
spinB10_action = AnimationAction(AnimationMixer(proStack10), spinB10_clip, proStack10)
spinB10_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [21]:
spinB11_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB11_clip = AnimationClip(tracks=[spinB11_track])
spinB11_action = AnimationAction(AnimationMixer(proStack11), spinB11_clip, proStack11)
spinB11_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [22]:
spinB12_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB12_clip = AnimationClip(tracks=[spinB12_track])
spinB12_action = AnimationAction(AnimationMixer(proStack12), spinB12_clip, proStack12)
spinB12_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [23]:
spinB13_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB13_clip = AnimationClip(tracks=[spinB13_track])
spinB13_action = AnimationAction(AnimationMixer(proStack13), spinB13_clip, proStack13)
spinB13_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [24]:
spinB14_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB14_clip = AnimationClip(tracks=[spinB14_track])
spinB14_action = AnimationAction(AnimationMixer(proStack14), spinB14_clip, proStack14)
spinB14_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [25]:
spinB15_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB15_clip = AnimationClip(tracks=[spinB15_track])
spinB15_action = AnimationAction(AnimationMixer(proStack15), spinB15_clip, proStack15)
spinB15_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [26]:
spinB16_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB16_clip = AnimationClip(tracks=[spinB16_track])
spinB16_action = AnimationAction(AnimationMixer(proStack16), spinB16_clip, proStack16)
spinB16_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [27]:
spinB17_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB17_clip = AnimationClip(tracks=[spinB17_track])
spinB17_action = AnimationAction(AnimationMixer(proStack17), spinB17_clip, proStack17)
spinB17_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [28]:
spinB18_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB18_clip = AnimationClip(tracks=[spinB18_track])
spinB18_action = AnimationAction(AnimationMixer(proStack18), spinB18_clip, proStack18)
spinB18_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…

In [29]:
spinB19_track = NumberKeyframeTrack(name='.rotation[z]', times=[0, 2], values=[0, 6.28])
spinB19_clip = AnimationClip(tracks=[spinB19_track])
spinB19_action = AnimationAction(AnimationMixer(proStack19), spinB19_clip, proStack19)
spinB19_action

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[z]', times=array([0, 2]), value…